<a href="https://colab.research.google.com/github/cognitedata/WiDS-2019/blob/master/WiDS_2019_Cognite_Interact_with_Assets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Visualization and Time series prediction

## What this notebook will achieve

* Extract live data from an oil rig in the North Sea.

* Visualize and inspect data directly from the Cognite Data Platform.

* Apply Linear Regression for time series prediction.


## Getting started

* Having a basic understanding of Python concepts will help to understand the process.

* Cognite has released *live* data to the public on the Cognite Data Platform streaming from [Valhall](https://www.akerbp.com/en/our-assets/production/valhall/), one of Aker's oil fields.

* To access the data, generate an API Key on [Open Industrial Data](https://openindustrialdata.com/). Get your key via the Google Access platform. You will be asked to fill out some personal information to generate your personal key.

* Visualize some of the machines (assets) on Valhall with Cognite's [Operational Intelligence](https://opint.cogniteapp.com/publicdata/infographics/-LOHKEJPLvt0eRIZu8mE) dashboard. This data on this page shows is streaming live data from the Valhall oil field located in the North Sea.

* To understand how to interact with the data using the Python SDK ([Docs](https://cognite-sdk-python.readthedocs-hosted.com/en/latest/)) follow along in this notebook.

## Environment Setup

#### Install the Cognite SDK package

In [ ]:
!pip install cognite-sdk

#### Import the required packages

In [1]:
%matplotlib notebook

import os
from datetime import datetime, timedelta
from datetime import datetime
from getpass import getpass

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from cognite.client.cognite_client import CogniteClient

#### Connect to the Cognite Data Platform
* This client object is how all queries will be sent to the Cognite API to retrieve data.

When prompted for your API key, use the key generated by open industrial data as mentioned in the Getting Started steps.

In [4]:
client = CogniteClient(api_key=getpass("Open Industrial Data API-KEY: "))

Open Industrial Data API-KEY: ········


## Accessing Cognite Data Platform (CDP)
* The CDP organizes digital information about the physical world.
* There are 6 kinds of objects stored on the CDP. Each of these objects in the CDP are labelled with a unique ID. Information regarding a specific Asset, Event, etc are often retrieved using this ID.

  * [Assets](https://doc.cognitedata.com/api/0.5/#tag/Assets) are digital representations of physical objects or groups of objects, and assets are organized into an asset hierarchy. For example, an asset can represent a water pump which is part of a subsystem on an oil platform.
  
  * [Event](https://doc.cognitedata.com/api/0.5/#tag/Events) objects store complex information about multiple assets over a time period. For example, an event can describe two hours of maintenance on a water pump and some associated pipes.
  
  * A [File](https://doc.cognitedata.com/api/0.5/#tag/Files) stores a sequence of bytes connected to one or more assets. For example, a file can contain a piping and instrumentation diagram (P&IDs) showing how multiple assets are connected.
  
  * A [Time Series](https://doc.cognitedata.com/api/0.5/#tag/Time-series) consists of a sequence of data points connected to a single asset. For example: A water pump asset can have a temperature time series that records a data point in units of °C every second.
  
  * [Sequence](https://doc.cognitedata.com/api/0.5/#tag/Sequences) are similar to time series in that they are a key value pair, but rather than using a timestamp as the key, another measurment such as depth could be the key. For example, this is used in practice when drilling and taking measurments at various depths.
  
  * A [3D](https://doc.cognitedata.com/api/0.5/#tag/3D) model is typically built up by a hierarchical structure. This looks very similar to how we organize our internal asset hierarchy. 3D models are visualized via Cognite's dashboards.
  
* It is important to refer back to the [SDK](https://cognite-sdk-python.readthedocs-hosted.com/en/latest/cognite.html) for specific details on arguments on all avaiable methods on how to access these objects.

### Collecting Asset Information

#### Retrieve a list of all Assets

* There are thousands of Assets in the CDP, we can have a look at a few examples.

* This will generate a list of assets from the CDP with no particular filters, this is a random result. Generally we would want to apply filters when retrieving records.


In [5]:
client.assets.get_assets().to_pandas().head()

,createdTime,depth,description,id,lastUpdatedTime,metadata,name,parentId,path
0,0,2,GAS COMPRESSION AND RE-INJECTION (PH),3111454725058294,0,"{'SOURCE_DB': 'workmate', 'SOURCE_TABLE': 'wma...",23,4650652196144007,"[6687602007296940, 4650652196144007, 311145472..."
1,0,7,1ST STAGE COMP DRY GAS SEAL SYS ON PH,3904753668320840,0,"{'ELC_STATUS_ID': '1211', 'SOURCE_DB': 'workma...",23-1ST STAGE COMP DRY GAS SEAL SYS-PH,4856008121737468,"[6687602007296940, 4650652196144007, 311145472..."
2,0,7,1ST STAGE COMP ENCLOSURE ON PH,2499711953216311,0,"{'ELC_STATUS_ID': '1211', 'SOURCE_DB': 'workma...",23-1ST STAGE COMP ENCLOSURE-PH,4856008121737468,"[6687602007296940, 4650652196144007, 311145472..."
3,0,7,1ST STAGE COMP LUBE OIL SYS ON PH,2137557577165478,0,"{'ELC_STATUS_ID': '1211', 'SOURCE_DB': 'workma...",23-1ST STAGE COMP LUBE OIL SYS-PH,4856008121737468,"[6687602007296940, 4650652196144007, 311145472..."
4,0,4,1ST STAGE COMPRESSION ON PH,4518112062673878,0,"{'ELC_STATUS_ID': '1211', 'SOURCE_DB': 'workma...",23-1ST STAGE COMPRESSION-PH,6895991969886325,"[6687602007296940, 4650652196144007, 311145472..."


#### Decide on which asset we want to explore
* To get started exploring data in the CDP, we must first decide on which Asset we want to gather information from.

* Some asset names may be retrieved from the [Op Int](https://opint.cogniteapp.com/publicdata/infographics/-LOHKEJPLvt0eRIZu8mE) dashboard.

* Here is a screehshot of the [OpInt Dashboard](https://drive.google.com/open?id=1f_7nJaJu5Xgr3Oq09mIZ0KwjBAYbzEUQ) incase the page does not load.

* Some example asset names are:
  * 23-HA-9103
  * 23-PV-92583
  * 23-VG-9101
  
The *fuzzy* search for an asset can be performed as followed


In [6]:
asset_name = "23-HA-9103"
asset_df = client.assets.search_for_assets(name=asset_name).to_pandas()
asset_df.head()


,createdTime,depth,description,id,lastUpdatedTime,metadata,name,parentId,path
0,0,6,VRD - 1ST STAGE SUCTION COOLER,2861239574637735,0,"{'ELC_STATUS_ID': '1211', 'RES_ID': '531306', ...",23-HA-9103,2513266419866445,"[6687602007296940, 4650652196144007, 311145472..."
1,0,8,VRD - 1ST STAGE COMPRESSOR LUBE OIL FILTER B,379497117972793,0,"{'ELC_STATUS_ID': '1211', 'RES_ID': '746776', ...",23-CB-9103B,2137557577165478,"[6687602007296940, 4650652196144007, 311145472..."
2,0,8,VRD - 1ST STAGE COMPPRESSOR LUBE OIL RESERVOIR,2357112351749647,0,"{'ELC_STATUS_ID': '1211', 'SOURCE_DB': 'workma...",23-TX-9103,2137557577165478,"[6687602007296940, 4650652196144007, 311145472..."
3,0,8,VRD - 1ST STAGE COMPRESSOR SEAL GAS PRESSURE B...,3089052537026304,0,"{'ELC_STATUS_ID': '1211', 'RES_ID': '476494', ...",23-KB-9103,3904753668320840,"[6687602007296940, 4650652196144007, 311145472..."
4,0,8,VRD - 1ST STAGE COMPRESSOR LUBE OIL FILTER A,8790823573167638,0,"{'ELC_STATUS_ID': '1211', 'RES_ID': '746776', ...",23-CB-9103A,2137557577165478,"[6687602007296940, 4650652196144007, 311145472..."


#### Get information on the asset of interest

* We can filter the assets to get asset-specific details based on asset_name

* The *get_asset()* interface provides the same information for 1 specific asset based on the provided ID


In [7]:
asset_id = asset_df[asset_df["name"] == asset_name].iloc[0]['id']
asset = client.assets.get_asset(asset_id=asset_id).to_pandas()
asset

,0
id,2861239574637735
depth,6
name,23-HA-9103
parentId,2513266419866445
description,VRD - 1ST STAGE SUCTION COOLER
metadata,"{'ELC_STATUS_ID': '1211', 'RES_ID': '531306', ..."
createdTime,0
lastUpdatedTime,0
path,"[6687602007296940, 4650652196144007, 311145472..."


#### How do we get Asset relationships?

* The interface *get_asset_subtree()* can be used to retrieve the *children* of an Asset. 

* Each Asset is given various properties, some of the useful ones for this method are:

  * Depth: The number of edges from the parent node
  
  * Description: Includes information such as the platform and type of sensor being monitored
  
We will generate a list of all children of the main asset of interest. This is done by specifying a depth of 1.

In [8]:
subtree_df = client.assets.get_asset_subtree(asset_id=asset_id, depth=1).to_pandas()
subtree_df.head()

,createdTime,depth,description,id,lastUpdatedTime,metadata,name,parentId,path
0,0,6,VRD - 1ST STAGE SUCTION COOLER,2861239574637735,0,"{'ELC_STATUS_ID': '1211', 'RES_ID': '531306', ...",23-HA-9103,2513266419866445,"[6687602007296940, 4650652196144007, 311145472..."
1,0,7,VRD - PH 1STSTGSUCTCOOL SHELL PSV IN,274450897701725,0,"{'ELC_STATUS_ID': '1225', 'RES_ID': '444134', ...",45-HV-92510-01,2861239574637735,"[6687602007296940, 4650652196144007, 311145472..."
2,0,7,VRD - PH 1STSTGSUCTCLR GAS IN,576308321452985,0,"{'ELC_STATUS_ID': '1211', 'RES_ID': '609895', ...",23-ESDV-92501A,2861239574637735,"[6687602007296940, 4650652196144007, 311145472..."
3,0,7,VRD - PH 1STSTGSUCTCOOL SHELL PSV OUT,619750565594754,0,"{'ELC_STATUS_ID': '1225', 'RES_ID': '510103', ...",45-HV-92510-03,2861239574637735,"[6687602007296940, 4650652196144007, 311145472..."
4,0,7,VRD - PH 1STSTGSUCTCOOL COOLMED OUT,705952550422793,0,"{'ELC_STATUS_ID': '1211', 'RES_ID': '485917', ...",45-PT-92508,2861239574637735,"[6687602007296940, 4650652196144007, 311145472..."


### Collecting Time Series Information and Data Points

* Time Series objects contain the bulk of information in the CDP.

* Time Series objects are generally linked to an asset through the asset_id field.

* The *get_time_series()* method has a variety of filters, all are listed in the SDK.

* By specifying **asset_id** we will get a dataframe of all the associated time series objects with that asset, this is seen below.

#### Compile a list of time series objects under the asset
* For each of the children assets, get the associated time series objects and merge them into a DataFrame

In [11]:
all_ts_df = pd.DataFrame()

for index, asset in subtree_df.iterrows():
    ts_df = client.time_series.get_time_series(asset_id=int(asset['id'])).to_pandas()
    if not ts_df.empty:
        all_ts_df = all_ts_df.append(ts_df, ignore_index=True)
all_ts_df

,assetId,createdTime,description,id,isStep,isString,lastUpdatedTime,name
0,705952550422793,0,PH 1stStgSuctCool CoolMed Out,3529821833330815,False,False,0,VAL_45-PT-92508:X.Value
1,786220428505816,0,PH 1stStgSuctCool Gas In,3518012501014915,False,False,0,VAL_23-FT-92512:X.Value
2,2814662602621825,0,PH 1stStgSuctCool Gas In,6156871056679530,False,False,0,VAL_23-PT-92512:X.Value
3,4840206559741735,0,PH 1stStgSuctCool Gas In,5474031062875475,False,False,0,VAL_23-TT-92502:X.Value
4,5193727763959013,0,PH 1stStgSuctCool Gas Out,7491681405625794,False,False,0,VAL_23-PT-92504:X.Value
5,5439867226448359,0,PH 1stStgSuctCool Gas In,988967451935968,False,False,0,VAL_23-TT-92512:X.Value
6,5497674254221099,0,PH 1stStgSuctCool CoolMed In,1920137775628302,False,False,0,VAL_45-FT-92506:X.Value
7,5552927149248373,0,PH 1stStgSuctCool Gas In ESDV,5880632484472759,False,False,0,VAL_23-PDT-92501:X.Value
8,5891006566061532,0,PH 1stStgSuctClr CoolMed Out,3929156348065703,False,False,0,VAL_45-TT-92508:X.Value
9,6681756879590530,0,PH 1stStgSuctCool Tubes,7891376783606949,False,False,0,VAL_23-PDT-92502:X.Value


#### View datapoints for one of the time series

* Now that we have a list of all of the time series directly related to the immediete childern of our asset, we can retrieve datapoints for some of the them.

* A Datapoint in the CDP is stored as a key value pair

  * timestamp is the time since epoch in milliseconds
  
  * value is the reading from the sensor
  
The identifier to retrieve Datapoints is the **name** column from the DataFrame above.

In [12]:
client.datapoints.get_datapoints("VAL_45-PT-92508:X.Value", start="10d-ago").to_pandas().head()

,timestamp,value
0,1553340692808,2.197147
1,1553340693808,2.204669
2,1553340694808,2.207782
3,1553340695808,2.198184
4,1553340696808,2.186511
